In [2]:
import pandas as pd
beta_ = 1 # parameter for f-betta measure

### загрузка данных

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train.columns

Index(['_id', 'age', 'job', 'marital', 'education', 'default', 'housing',
       'loan', 'contact', 'month', 'day_of_week', 'duration', 'campaign',
       'pdays', 'previous', 'poutcome', 'emp.var.rate', 'cons.price.idx',
       'cons.conf.idx', 'euribor3m', 'nr.employed', 'target'],
      dtype='object')

In [5]:
test.head() #x for x in train.columns if x not in test.columns

,_id,age,job,marital,education,default,housing,loan,contact,month,...,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
0,66810d8e6bf2b41c880a7bc6c8a1e295,57,technician,married,high.school,no,no,yes,cellular,may,...,371,1,999,1,failure,-1.8,92.893,-46.2,1.299,5099.1
1,ccac3879652b08cb8b44c1920fd93afa,55,unknown,married,unknown,unknown,yes,no,telephone,may,...,285,2,999,0,nonexistent,1.1,93.994,-36.4,4.860,5191.0
2,fcccab4d7a76f70647f015f2c84c2af8,33,blue-collar,married,basic.9y,no,no,no,cellular,may,...,52,1,999,1,failure,-1.8,92.893,-46.2,1.313,5099.1
3,ed8399278c30678dab739045fa12b440,36,admin.,married,high.school,no,no,no,telephone,jun,...,355,4,999,0,nonexistent,1.4,94.465,-41.8,4.967,5228.1
4,1d4d62ac5cabcb48bac7112813f290cb,27,housemaid,married,high.school,no,yes,no,cellular,jul,...,189,2,999,0,nonexistent,1.4,93.918,-42.7,4.963,5228.1


In [6]:
# number of 0's and 1's in target
train.loc[:, 'target'].value_counts()

0    21925
1     2787
Name: target, dtype: int64

In [7]:
train.loc[:, 'sample'] = 'train'
test.loc[:, 'sample'] = 'test'

In [8]:
df = test.append(train).reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 23 columns):
_id               41188 non-null object
age               41188 non-null int64
campaign          41188 non-null int64
cons.conf.idx     41188 non-null float64
cons.price.idx    41188 non-null float64
contact           41188 non-null object
day_of_week       41188 non-null object
default           41188 non-null object
duration          41188 non-null int64
education         41188 non-null object
emp.var.rate      41188 non-null float64
euribor3m         41188 non-null float64
housing           41188 non-null object
job               41188 non-null object
loan              41188 non-null object
marital           41188 non-null object
month             41188 non-null object
nr.employed       41188 non-null float64
pdays             41188 non-null int64
poutcome          41188 non-null object
previous          41188 non-null int64
sample            41188 non-null object
target     

### типы исходных данных и заполняемость

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24712 entries, 0 to 24711
Data columns (total 23 columns):
_id               24712 non-null object
age               24712 non-null int64
job               24712 non-null object
marital           24712 non-null object
education         24712 non-null object
default           24712 non-null object
housing           24712 non-null object
loan              24712 non-null object
contact           24712 non-null object
month             24712 non-null object
day_of_week       24712 non-null object
duration          24712 non-null int64
campaign          24712 non-null int64
pdays             24712 non-null int64
previous          24712 non-null int64
poutcome          24712 non-null object
emp.var.rate      24712 non-null float64
cons.price.idx    24712 non-null float64
cons.conf.idx     24712 non-null float64
euribor3m         24712 non-null float64
nr.employed       24712 non-null float64
target            24712 non-null int64
sample      

In [10]:
for i in train.columns: # перебираем все столбцы
    if (str(train[i].dtype) == 'object') & (i!='_id'): # если тип столбца - object
        print('='*10)
        print(i) # выводим название столбца
        print(set(train[i])) # выводим все его значения (но делаем set - чтоб значения не повторялись)
        print('\n') # выводим пустую строку

job
{'services', 'unemployed', 'admin.', 'technician', 'entrepreneur', 'blue-collar', 'self-employed', 'housemaid', 'student', 'retired', 'management', 'unknown'}


marital
{'married', 'unknown', 'divorced', 'single'}


education
{'unknown', 'illiterate', 'high.school', 'basic.4y', 'basic.9y', 'professional.course', 'basic.6y', 'university.degree'}


default
{'yes', 'unknown', 'no'}


housing
{'yes', 'unknown', 'no'}


loan
{'yes', 'unknown', 'no'}


contact
{'telephone', 'cellular'}


month
{'oct', 'aug', 'sep', 'jun', 'may', 'mar', 'dec', 'jul', 'nov', 'apr'}


day_of_week
{'fri', 'thu', 'mon', 'tue', 'wed'}


poutcome
{'success', 'nonexistent', 'failure'}


sample
{'train'}




### обработка категориальных признаков

In [11]:
from collections import Counter # Не считать же частоты самим.
Counter(train['day_of_week'].values)

for_map = [{'illiterate':1, 'basic.4y':2, 'basic.6y':3, \
    'basic.9y':4, 'high.school':5, 'professional.course':6, 'university.degree':7, 'unknown':-10}, \
{'mar': 3, 'apr': 4, \
    'may': 5, 'jun': 6, 'jul': 7, 'aug': 8, 'sep': 9, 'oct': 10, 'nov': 11, 'dec': 12}, \
{'fri': 5, 'mon': 1, 'thu': 2, 'tue': 4, 'wed': 3}]
column_order = ['education', 'month', 'day_of_week']

column_OneHot = ['job', 'marital', 'default', 'housing', 'loan', 'contact', 'poutcome']

column_ids = ['_id']

In [12]:
## from sklearn import preprocessing 
from sklearn.preprocessing import RobustScaler, PolynomialFeatures
from sklearn import preprocessing 

def preproc_data(df_input, column_order, value_order, column_OneHot, column_ids):
    df_output = df_input.copy()    
    
    df_output = df_output.drop(columns=column_ids, axis=1)
    
    # Recoding categorical
    for i in range(len(column_order)):
        df_output[column_order[i]] = df_output[column_order[i]].map(value_order[i])
    
    # OneHotEncoder other categorical
    le = preprocessing.LabelEncoder()
    # FIT AND TRANSFORM LabelEncoder    
    X_2 = df_output[column_OneHot].apply(le.fit_transform)
    
    enc = preprocessing.OneHotEncoder()
    # FIT OneHotEncoder
    enc.fit(X_2)
    # Transform
    onehotlabels = enc.transform(X_2).toarray()
    
    # concat Numerical features and OneHot-Categorical    
    X_3 = df_output[[x for x in df_output.columns if x not in column_OneHot]]
    
    X_4 = pd.DataFrame(onehotlabels)
       
    for c in X_4.columns:
        X_3[c] = X_4[c].values 
    
    df_output = X_3
    return df_output


def scaler_data(df_input):
    df_output = df_input.copy() 
    scaler = RobustScaler()
    scaler.fit(df_output)    
    df_output = scaler.transform(df_output) # http://localhost:8888/notebooks/Google%20Drive/SkillFactory/HW_lesson_03/hw-03.ipynb#    
    return df_output

In [13]:
# put the last element to the first position
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

In [14]:
df_preproc = df.pipe(preproc_data, column_order, for_map, column_OneHot, column_ids)

df_train_preproc = df_preproc.query('sample == "train"').drop(['sample'], axis=1)
df_test_preproc = df_preproc.query('sample == "test"').drop(['sample'], axis=1)


C:\Users\skulivec\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [15]:
df_train_preproc.columns

Index([        'target',            'age',       'campaign',  'cons.conf.idx',
       'cons.price.idx',    'day_of_week',       'duration',      'education',
         'emp.var.rate',      'euribor3m',          'month',    'nr.employed',
                'pdays',       'previous',                0,                1,
                      2,                3,                4,                5,
                      6,                7,                8,                9,
                     10,               11,               12,               13,
                     14,               15,               16,               17,
                     18,               19,               20,               21,
                     22,               23,               24,               25,
                     26,               27,               28,               29],
      dtype='object')

#### Рисует зависимость предикторов друг от друга
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(style="white")

df = df_train_preproc[[        'target',            'age',       'campaign',  'cons.conf.idx',
       'cons.price.idx',    'day_of_week',       'duration',      'education',
         'emp.var.rate',      'euribor3m',          'month',    'nr.employed',
                'pdays',       'previous',                0,                1,
                      2,                3,                4,                5,
                      6,                7,                8,                9,
                     10,               11,               12,               13,
                     14,               15,               16,               17,
                     18,               19,               20,               21,
                     22,               23,               24,               25,
                     26,               27,               28,               29]]
print('df.columns')
print(df.columns)
g = sns.PairGrid(df, diag_sharey=False, hue="target")
g = g.map_diag(plt.hist)
g = g.map_offdiag(plt.scatter)
g = g.add_legend()

##### выделение предикторов и целевых переменных

In [17]:
X, y = df_train_preproc.iloc[:, 1:].values, df_train_preproc.iloc[:, 0].values
X_test, y_test = df_test_preproc.iloc[:, 1:].values, df_test_preproc.iloc[:, 0].values


#### полиномиальные фичи + приведение шкал

In [18]:
#poly = PolynomialFeatures(2)

#X = poly.fit_transform(X)
#X_test = poly.fit_transform(X_test)

X = scaler_data(X)
X_test = scaler_data(X_test)



In [19]:
X_, y_ = X, y # сохраняем для обучения модели в конце

#### выделяем validate

In [20]:
random_state=[42, 123, 0]
test_size=0.3

from sklearn.model_selection import train_test_split
X, X_validate, y, y_validate = train_test_split(X_, y_, test_size=test_size, random_state=random_state[0])

In [21]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, fbeta_score, average_precision_score

# Logistic Regression

import matplotlib.pyplot as plt
%matplotlib inline
import random

c_range = np.logspace(-3, 3, 10) # from 10^(-3)...10^3 with only 10 values
scores=[]
for C in c_range:
    X, X_validate, y, y_validate = train_test_split(X_, y_, test_size=test_size, random_state=random_state) #random.randint(1, 1000))
    model = LogisticRegression(penalty='l2', fit_intercept=True, C=C, random_state=random_state, n_jobs=-1)
    model.fit(X, y)   
    y_lr_proba = model.predict_proba(X_validate)[:,1]
    y_lr = model.predict(X_validate)
    scores.append(fbeta_score(y_validate,y_lr,beta=beta_))
plt.plot(c_range, scores)

c_range[scores==max(scores)][0]

In [22]:
y_lr_proba=[]
y_lr=[]

X_validate_2 = X_validate
X_2 = X
X__2 = X_
X_test_2 = X_test

### Подбор параметра регрессии
C=[0.06579332246575682, 0.004641588833612777, 0.1] #c_range[scores==max(scores)][0] # 0.06579332246575682   0.004641588833612777
print('X_validate_2.shape')
print(X_validate_2.shape)
for i in range(len(random_state)):
    model = LogisticRegression(penalty='l2', fit_intercept=True, C=C[i], random_state=random_state[i], n_jobs=-1)
    model.fit(X, y)
    y_lr_proba = model.predict_proba(X_validate)[:,1]
    y_lr = model.predict(X_validate)
    
    
    X_validate_2 = np.c_[ X_validate_2, model.predict_proba(X_validate)[:,1] ]  
    X_2 = np.c_[ X_2, model.predict_proba(X)[:,1] ]
    
    model.fit(X_, y_)
    X__2 = np.c_[ X__2, model.predict_proba(X_)[:,1] ]        
    X_test_2 = np.c_[ X_test_2, model.predict_proba(X_test)[:,1] ]
    
    print('LogisticRegression on Validate (Prec, Recall, Roc, Acc):')
    print('Fbeta', fbeta_score(y_validate,y_lr,beta=beta_))
    print('Prec', precision_score(y_validate,y_lr))
    print('Recall', recall_score(y_validate, y_lr))
    print('Roc', roc_auc_score(y_validate, y_lr_proba)) 
    print('Acc', accuracy_score(y_validate, y_lr))
    print('ARS', average_precision_score(y_validate, y_lr))
    print('X_validate_2.shape')
    print(X_validate_2.shape)

####### pow 1 (C=0.06579332246575682)
LogisticRegression on Validate (Prec, Recall, Roc, Acc):
Fbeta 0.4416243654822335
Prec 0.6796875
Recall 0.40606767794632437
Roc 0.9224928012123824
Acc 0.9092257890477475

####### pow 2 (C=0.06579332246575682)
LogisticRegression on Validate (Prec, Recall, Roc, Acc):
Fbeta 0.48281092012133464
Prec 0.6541095890410958
Recall 0.4531435349940688
Roc 0.9290409908882861
Acc 0.9105745886161316

LogisticRegression on Validate (Prec, Recall, Roc, Acc): C=0.46415888336127775
Fbeta 0.44952478808117136
Prec 0.6717850287907869
Recall 0.4151838671411625
Roc 0.9256794069632318
Acc 0.9104397086592932

LogisticRegression on Validate (Prec, Recall, Roc, Acc): C=0.004641588833612777
Fbeta 0.43762833675564683
Prec 0.6507633587786259
Recall 0.4045077105575326
Roc 0.9245446173948473
Acc 0.9076072295656865

####### pow 3 (C=0.06579332246575682)
LogisticRegression on Validate (Prec, Recall, Roc, Acc):
Fbeta 0.44038805208067405
Prec 0.6330275229357798
Recall 0.4092526690391459
Roc 0.9243077666290631
Acc 0.9058537901267871

In [ ]:
#### learn model for all train dataset
model.fit(X_, y_)

In [ ]:
vals=model.predict(X_test)

pd.DataFrame({'_id':test['_id'], 'target':[int(i) for i in vals]}).to_csv('sample_submission.csv', index = False)

# Decision tree

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as randint
from sklearn.model_selection import StratifiedKFold

##### Подбор оптимальных параметров модели на кросс-валидации 
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(4, 8),
    'min_samples_leaf': randint(4, 8),
    'class_weight': [None, 'balanced']}


##### Будем делать 400 запусков поиска
cv = StratifiedKFold(n_splits=5, random_state=random_state, shuffle=True)

model = DecisionTreeClassifier(random_state=random_state)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=400, n_jobs=-1,
                                   cv=cv, scoring='f1', random_state=random_state) # , 'precision', 'f1', 'recall', 'roc_auc'
random_search.fit(X, y)
print(random_search.scoring)
print('random_search.best_params_')
print(random_search.best_params_)
print('random_search.best_score_')
print(random_search.best_score_)
print(random_search.best_estimator_)
print('random_search.best_estimator_')

WITHOUT Polinomial
f1
random_search.best_params_
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 5}
random_search.best_score_
0.5623051853546034
DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
            
precision
random_search.best_params_
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 5}
random_search.best_score_
0.6466089479600076
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
            
roc_auc
random_search.best_params_
{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 7}
random_search.best_score_
0.9294953638394235
DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=5, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
            
accuracy
random_search.best_params_
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 5}
random_search.best_score_
0.911955139322465
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
            

average_precision
random_search.best_params_
{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 6}
random_search.best_score_
0.6015547065646177
DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=6, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=6, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
            
WITH Polinomial
f1
random_search.best_params_
{'class_weight': 'balanced', 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 4}
random_search.best_score_
0.5806672468481308
DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

precision
random_search.best_params_
{'class_weight': None, 'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 4}
random_search.best_score_
0.6517374731505117
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
            
roc_auc
random_search.best_params_
{'class_weight': 'balanced', 'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 4}
random_search.best_score_
0.926554480351135
DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=4, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
            
accuracy
random_search.best_params_
{'class_weight': None, 'criterion': 'gini', 'max_depth': 5, 'min_samples_leaf': 4}
random_search.best_score_
0.9123598103827032
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=4, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

average_precision
random_search.best_params_
{'class_weight': None, 'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 7}
random_search.best_score_
0.5971769349065488
DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=7, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')

In [25]:
# max precision
model = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
model.fit(X, y)
y_lr_proba = model.predict_proba(X_validate)[:,1]
y_lr = model.predict(X_validate)
print('DecisionTreeClassifier on Validate:')
print('precision')
print('Fbeta', fbeta_score(y_validate,y_lr,beta=beta_))
print('Prec', precision_score(y_validate,y_lr))
print('Recall', recall_score(y_validate, y_lr))
print('Roc', roc_auc_score(y_validate, y_lr_proba)) 
print('Acc', accuracy_score(y_validate, y_lr))
print('ARS', average_precision_score(y_validate, y_lr))

# max recall
model = DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=6, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
model.fit(X, y)
y_lr_proba = model.predict_proba(X_validate)[:,1]
y_lr = model.predict(X_validate)
print('DecisionTreeClassifier on Validate:')
print('recall')
print('Fbeta', fbeta_score(y_validate,y_lr,beta=beta_))
print('Prec', precision_score(y_validate,y_lr))
print('Recall', recall_score(y_validate, y_lr))
print('Roc', roc_auc_score(y_validate, y_lr_proba)) 
print('Acc', accuracy_score(y_validate, y_lr))
print('ARS', average_precision_score(y_validate, y_lr))

# max f1
model = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=9, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
model.fit(X, y)
y_lr_proba = model.predict_proba(X_validate)[:,1]
y_lr = model.predict(X_validate)
print('DecisionTreeClassifier on Validate:')
print('f1')
print('Fbeta', fbeta_score(y_validate,y_lr,beta=beta_))
print('Prec', precision_score(y_validate,y_lr))
print('Recall', recall_score(y_validate, y_lr))
print('Roc', roc_auc_score(y_validate, y_lr_proba)) 
print('Acc', accuracy_score(y_validate, y_lr))
print('ARS', average_precision_score(y_validate, y_lr))

# max roc_auc
model = DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
            max_depth=6, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=9, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=42,
            splitter='best')
model.fit(X, y)
y_lr_proba = model.predict_proba(X_validate)[:,1]
y_lr = model.predict(X_validate)
print('DecisionTreeClassifier on Validate:')
print('roc_auc')
print('Fbeta', fbeta_score(y_validate,y_lr,beta=beta_))
print('Prec', precision_score(y_validate,y_lr))
print('Recall', recall_score(y_validate, y_lr))
print('Roc', roc_auc_score(y_validate, y_lr_proba)) 
print('Acc', accuracy_score(y_validate, y_lr))
print('ARS', average_precision_score(y_validate, y_lr))

# обучение модели на всём train dataset'е и сохранение результатов, т.к. это лучшая модель
model.fit(X_, y_)
vals=model.predict(X_test)
pd.DataFrame({'_id':test['_id'], 'target':[int(i) for i in vals]}).to_csv('sample_submission.csv', index = False)

DecisionTreeClassifier on Validate:
precision
Fbeta 0.5744125326370757
Prec 0.6386066763425254
Recall 0.5219454329774614
Roc 0.9193076339420869
Acc 0.9120582681413542
ARS 0.38767446079177775
DecisionTreeClassifier on Validate:
recall
Fbeta 0.5009475679090335
Prec 0.34136891950064574
Recall 0.9406880189798339
Roc 0.9229623929003984
Acc 0.7868896681953061
ARS 0.32786565046826954
DecisionTreeClassifier on Validate:
f1
Fbeta 0.6009445100354192
Prec 0.5981198589894242
Recall 0.6037959667852907
Roc 0.9029034618302896
Acc 0.9088211491772322
ARS 0.4061922640960313
DecisionTreeClassifier on Validate:
roc_auc
Fbeta 0.5463483146067417
Prec 0.38802992518703244
Recall 0.9228944246737841
Roc 0.9319814064927798
Acc 0.8257350957647693
ARS 0.3668778517561947


# Попытка сделать стекинг моделей

for i in range(len(random_state)):
    print('random_state[i]')
    print(random_state[i])
    model = DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=random_state[i],
            splitter='best')
    model.fit(X, y)
    y_lr_proba = model.predict_proba(X_validate)[:,1]
    y_lr = model.predict(X_validate)
    
    X_validate_2 = np.c_[ X_validate_2, model.predict_proba(X_validate)[:,1] ]  
    X_2 = np.c_[ X_2, model.predict_proba(X)[:,1] ]
    
    model.fit(X_, y_)
    X__2 = np.c_[ X__2, model.predict_proba(X_)[:,1] ]        
    X_test_2 = np.c_[ X_test_2, model.predict_proba(X_test)[:,1] ]
       
    print('DecisionTreeClassifier on Validate:')
    print('Fbeta', fbeta_score(y_validate,y_lr,beta=beta_))
    print('Prec', precision_score(y_validate,y_lr))
    print('Recall', recall_score(y_validate, y_lr))
    print('Roc', roc_auc_score(y_validate, y_lr_proba)) 
    print('Acc', accuracy_score(y_validate, y_lr))
    print('ARS', average_precision_score(y_validate, y_lr))
    print('X_validate_2.shape')
    print(X_validate_2.shape)       


model = DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=random_state[0],
            splitter='best')

model.fit(X_2, y)
y_lr_proba = model.predict_proba(X_validate_2)[:,1]
y_lr = model.predict(X_validate_2)

print('DecisionTreeClassifier-2 on Validate:')
print('Fbeta', fbeta_score(y_validate,y_lr,beta=beta_))
print('Prec', precision_score(y_validate,y_lr))
print('Recall', recall_score(y_validate, y_lr))
print('Roc', roc_auc_score(y_validate, y_lr_proba)) 
print('Acc', accuracy_score(y_validate, y_lr))
print('ARS', average_precision_score(y_validate, y_lr))

======= globe 2 ===
DecisionTreeClassifier-2 on Validate:
Fbeta 0.5751685838952797
Prec 0.43206197854588796
Recall 0.8600237247924081
Roc 0.8814618783096148
Acc 0.8555435662260588
ARS 0.38749938703714487

===============

without polinomial f1
DecisionTreeClassifier on Validate:
Fbeta 0.7230856709628507
Prec 0.4007352941176471
Recall 0.9051008303677343
Roc 0.9134765377833838
Acc 0.8353115727002968

with polinomial f1
DecisionTreeClassifier on Validate:
Fbeta 0.7291265871489034
Prec 0.41511500547645125
Recall 0.8991696322657177
Roc 0.925844769235685
Acc 0.8444834097653089

with polinomial precision
DecisionTreeClassifier on Validate:
Fbeta 0.518927049385811
Prec 0.6709886547811994
Recall 0.49110320284697506
Roc 0.9092042879375984
Acc 0.9147558672781225

with polinomial average_precision
DecisionTreeClassifier on Validate:
Fbeta 0.6154747948417351
Prec 0.5879059350503919
Recall 0.6227758007117438
Roc 0.9173144408742321
Acc 0.9074723496088482
ARS 0.4090254157188098

model.fit(X__2, y_)
vals=model.predict(X_test_2)
pd.DataFrame({'_id':test['_id'], 'target':[int(i) for i in vals]}).to_csv('sample_submission.csv', index = False)

# k-means

In [29]:
from sklearn.neighbors import KNeighborsClassifier

##### Подбор оптимальных параметров модели на кросс-валидации 
param_grid = {
    'n_neighbors': randint(3, 8),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': randint(20, 40),
    'p': randint(1,2)}
##### Будем делать 400 запусков поиска
cv = StratifiedKFold(n_splits=5, random_state=random_state, shuffle=True)

model = KNeighborsClassifier()
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=400, n_jobs=-1,
                                   cv=cv, scoring='f1', random_state=random_state) # 'f1', 'precision', 'recall',
                                                                                   # 'roc_auc', 'average_precision',
                                                                                   # 'accuracy', ''
random_search.fit(X, y)
print(random_search.scoring)
print('random_search.best_params_')
print(random_search.best_params_)
print('random_search.best_score_')
print(random_search.best_score_)
random_search.best_estimator_

In [30]:
model = KNeighborsClassifier(n_neighbors=5, weights='uniform', metric='minkowski', p=2, n_jobs=-1)
model.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform')

In [31]:
y_lr = model.predict(X_validate)
y_lr_proba = model.predict_proba(X_validate)[:,1]

In [32]:
print('KNeighborsClassifier on Validate:')
print('Fbeta', fbeta_score(y_validate,y_lr,beta=beta_))
print('Prec', precision_score(y_validate,y_lr))
print('Recall', recall_score(y_validate, y_lr))
print('Roc', roc_auc_score(y_validate, y_lr_proba)) 
print('Acc', accuracy_score(y_validate, y_lr))
print('ARS', average_precision_score(y_validate, y_lr))

KNeighborsClassifier on Validate:
Fbeta 0.4549828178694158
Prec 0.5408496732026143
Recall 0.3926453143534994
Roc 0.8441883014135405
Acc 0.8930401942271379
ARS 0.2814206278538958


In [ ]:
# обучение модели на всём train dataset'е
model.fit(X_, y_)
vals=model.predict(X_test)